**Creat and run Amazon Redshift Cluster using AWS SDK, IAC (Infrastructure-as-code)**

In [ ]:
import os
import boto3
import json
import pandas as pd
import psycopg2

**Create data warehouse parameters**

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

**Create clients for IAM, EC2, S3 and Redshift**

In [ ]:
import boto3

ec2 = boto3.resource('ec2',
                     region_name="us-west-2",
                     aws_access_key_id=KEY,
                     aws_secret_access_key=SECRET
                    )

s3 = boto3.resource('s3',
                    region_name="us-west-2",
                    aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET
                    )

iam = boto3.client('iam', aws_access_key_id=KEY,
                    aws_secret_access_key=SECRET,
                    region_name='us-west-2'
                  )

redshift = boto3.client('redshift',
                         region_name='us-west-2',
                         aws_access_key_id=KEY,
                         aws_secret_access_key=SECRET
                       )

**Investigate the data in the S3 bucket**

In [ ]:
bucket=s3.Bucket('udacity-dend')

def data_files(prefix):
    data_files = [o.key for o in bucket.objects.filter(Prefix=prefix)]
    print ("Number of files in data_files:",len(data_files)-1)

In [ ]:
log_files     = data_files('log_data')
song_files    = data_files('song-data/A/A')

**Create IAM role so Amazon Redshift is able to impersonate S3 bucket**

In [ ]:
from botocore.exceptions import ClientError

# create IAM role
try:
    print("create a new IAM role")
    dwhRole =iam.create_role(
                            Path                     = '/',
                            RoleName                 = DWH_IAM_ROLE_NAME,
                            Description              = "allow Redshift clusters to call AWS services on your behalf.",
                            AssumeRolePolicyDocument = json.dumps(
                            {'Statement': [{'Action': 'sts:AssumeRole',
                                           'Effect': 'Allow',
                                           'Principal': {'Service': 'redshift.amazonaws.com'}}],
                             'Version': '2012-10-17'
                            })
    )
except Exception as e:
    print(e)
    
print('attaching policy')
iam.attach_role_policy(RoleName = DWH_IAM_ROLE_NAME,
                       PolicyArn = "arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
                      )['ResponseMetadata']['HTTPStatusCode']

print("get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)['Role']['Arn']

print(roleArn)

**Create Redshift cluster**

In [ ]:
try:
    response = redshift.create_cluster(
        ClusterType        = DWH_CLUSTER_TYPE,
        NodeType           = DWH_NODE_TYPE,
        NumberOfNodes      = int(DWH_NUM_NODES),
        # identifiers & credentials
        DBName             = DWH_DB,
        ClusterIdentifier  = DWH_CLUSTER_IDENTIFIER,
        MasterUsername     = DWH_DB_USER,
        MasterUserPassword = DWH_DB_PASSWORD,
        # IAM role
        IamRoles = [roleArn]
    )
except Exception as e:
    print(e)

**Check on cluster status**

In [ ]:
def prettyRedshiftProps(props):
    pd.set_option('display.max_colwidth', -1)
    keysToShow = ["ClusterIdentifier", "NodeType", "ClusterStatus", "MasterUsername", "DBName", "Endpoint", "NumberOfNodes", 'VpcId']
    x = [(k, v) for k,v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])

myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)['Clusters'][0]
prettyRedshiftProps(myClusterProps)

**Record cluster endpoint and role ARN**

In [ ]:
DWH_ENDPOINT = myClusterProps['Endpoint']['Address']
DWH_ROLE_ARN = myClusterProps['IamRoles'][0]['IamRoleArn']
print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

**Open a TCP port to access the cluster**

In [ ]:
try:
    vpc = ec2.Vpc(id=myClusterProps['VpcId'])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp='0.0.0.0/0',
        IpProtocol='TCP',
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT)
    )
except Exception as e:
    print(e)

**Check on connection to the cluster**

In [ ]:
# FILL IN THE REDSHIFT ENPOINT HERE
# e.g. DWH_ENDPOINT="redshift-cluster-1.csmamz5zxmle.us-west-2.redshift.amazonaws.com" 
DWH_ENDPOINT="dwhcluster.cxhcehxfoqqg.us-west-2.redshift.amazonaws.com" 
    
#FILL IN THE IAM ROLE ARN you got in step 2.2 of the previous exercise
#e.g DWH_ROLE_ARN="arn:aws:iam::988332130976:role/dwhRole"
DWH_ROLE_ARN="arn:aws:iam::851744632117:role/dwhRole"

In [ ]:
conn_string="postgresql://{}:{}@{}:{}/{}".format(DWH_DB_USER, DWH_DB_PASSWORD, DWH_ENDPOINT, DWH_PORT,DWH_DB)
print(conn_string)
%sql $conn_string

In [ ]:
%run -i sql_queries.py

In [ ]:
%run -i create_tables.py

In [ ]:
%run -i etl.py